In [8]:
import pandas as pd
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense

data_path = '../sim/'

In [15]:
logs = pd.read_csv(data_path + 'driving_log.csv')

logs = logs.values

images = []
measurements = []

images_path = data_path + 'IMG/'

for line in logs:
    center_path = line[0]
    center_path = images_path + center_path.split('/')[-1]
    center_img = cv2.imread(center_path)
    images.append(center_img)
    
    measure = float(line[3])
    measurements.append(measure)
    
X_train = np.array(images)
y_train = np.array(measurements)
    
    

In [17]:
model = Sequential()

model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train)

model.save('model.h5')

Epoch 1/10
102/102 [==============================] - 0s 2ms/step - loss: 147752355.2426
Epoch 2/10
102/102 [==============================] - 0s 794us/step - loss: 151982039.9730
Epoch 3/10
102/102 [==============================] - 0s 794us/step - loss: 92253810.8235
Epoch 4/10
102/102 [==============================] - 0s 804us/step - loss: 19807960.0784
Epoch 5/10
102/102 [==============================] - 0s 804us/step - loss: 26475876.4412
Epoch 6/10
102/102 [==============================] - 0s 814us/step - loss: 28468094.2549
Epoch 7/10
102/102 [==============================] - 0s 833us/step - loss: 7273458.2745
Epoch 8/10
102/102 [==============================] - 0s 794us/step - loss: 6482896.8358
Epoch 9/10
102/102 [==============================] - 0s 824us/step - loss: 8091179.6434
Epoch 10/10
102/102 [==============================] - 0s 814us/step - loss: 3585877.1961
